In [2]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import oandapy
import configparser
import datetime
from datetime import datetime, timedelta

In [3]:
#自分のアカウント、トークンをセット
accountID = "101-009-12923052-001"
access_token = '7b8e8782a87ce72d52cd23b2d74772df-13011574ae44dc0db94fc8dac52e4ebc'


In [4]:
# デモアカウントでAPI呼び出し
oanda = oandapy.API(environment="live", 
                    access_token=ACCESS_TOKEN)

AttributeError: module 'oandapy' has no attribute 'API'

In [5]:
# ドル円 1日足 過去レートの取得
response = oanda.get_history(instrument="USD_JPY", granularity="D")

{'instrument': 'USD_JPY',
 'granularity': 'D',
 'candles': [{'complete': True,
   'volume': 31968,
   'time': '2018-04-03T21:00:00.000000000Z',
   'mid': {'o': '106.624', 'h': '106.848', 'l': '105.992', 'c': '106.777'}},
  {'complete': True,
   'volume': 21819,
   'time': '2018-04-04T21:00:00.000000000Z',
   'mid': {'o': '106.777', 'h': '107.492', 'l': '106.713', 'c': '107.385'}},
  {'complete': True,
   'volume': 40095,
   'time': '2018-04-05T21:00:00.000000000Z',
   'mid': {'o': '107.408', 'h': '107.460', 'l': '106.778', 'c': '106.927'}},
  {'complete': True,
   'volume': 21284,
   'time': '2018-04-08T21:00:00.000000000Z',
   'mid': {'o': '106.966', 'h': '107.203', 'l': '106.619', 'c': '106.780'}},
  {'complete': True,
   'volume': 29076,
   'time': '2018-04-09T21:00:00.000000000Z',
   'mid': {'o': '106.780', 'h': '107.398', 'l': '106.620', 'c': '107.202'}},
  {'complete': True,
   'volume': 23970,
   'time': '2018-04-10T21:00:00.000000000Z',
   'mid': {'o': '107.218', 'h': '107.254'

In [6]:
# dictからDataFrameへ変換
res = pd.DataFrame(response["candles"])
 
# 最初の5行を表示させる
res.head()
 

In [7]:
# 日付の調整する
# 文字列 -> datetime
def iso_jp(iso):
    date = None
    try:
        date = datetime.strptime(iso, '%Y-%m-%dT%H:%M:%S.%fZ')
        date = pytz.utc.localize(date).astimezone(pytz.timezone("Asia/Tokyo"))
    except ValueError:
        try:
            date = datetime.strptime(iso, '%Y-%m-%dT%H:%M:%S.%f%z')
            date = dt.astimezone(pytz.timezone("Asia/Tokyo"))
        except ValueError:
            pass
    return date
 
# datetime -> 表示用文字列
def date_string(date):
    if date is None:
        return ''
    return date.strftime('%Y/%m/%d %H:%M:%S')
 

In [8]:
# ISOから認識しやすい日付へ変換
res['time'] = res['time'].apply(lambda x: iso_jp(x))
res['time'] = res['time'].apply(lambda x: date_string(x))
 
res.head()

                           volume     open     high      low    close
time                                                                 
2020-02-06 22:00:00+00:00   68129  109.992  110.022  109.532  109.749
2020-02-09 22:00:00+00:00   61546  109.688  109.879  109.562  109.775
2020-02-10 22:00:00+00:00   45837  109.771  109.966  109.728  109.789
2020-02-11 22:00:00+00:00   44585  109.788  110.136  109.772  110.095
2020-02-12 22:00:00+00:00   42178  110.089  110.094  109.619  109.726


In [9]:
# データフレームのサイズを確認
print(res.shape)
print(res.columns)
 

(500, 5)

In [10]:
# 必要なデータへ切り分け（askのみ）
df = res[['time', 'openAsk', 'closeAsk', 'highAsk', 'lowAsk', 'volume']]
df.columns = ['time', 'open', 'close', 'high', 'low', 'volume']

Index(['volume', 'open', 'high', 'low', 'close'], dtype='object')

In [11]:
# データフレームの399件〜410件を表示
df[399:410]

,volume,open,high,low,close
time,,,,,
2018-04-03 21:00:00+00:00,31968,106.624,106.848,105.992,106.777
2018-04-04 21:00:00+00:00,21819,106.777,107.492,106.713,107.385
2018-04-05 21:00:00+00:00,40095,107.408,107.460,106.778,106.927
2018-04-08 21:00:00+00:00,21284,106.966,107.203,106.619,106.780
2018-04-09 21:00:00+00:00,29076,106.780,107.398,106.620,107.202
...,...,...,...,...,...
2019-09-19 21:00:00+00:00,16306,108.024,108.088,107.529,107.565
2019-09-20 21:00:00+00:00,1,107.554,107.554,107.554,107.554
2019-09-22 21:00:00+00:00,14691,107.718,107.769,107.310,107.546


In [12]:
# 訓練とテストで日付区切る
split_date = '2018/02/19 07:00:00'
train, test = df[df['time'] < split_date], df[df['time']>=split_date]
del train['time']
del test['time']

In [13]:
# 念のため確認
train.shape, test.shape

((194, 5), (275, 5))

In [14]:
# windowを設定
window_len = 10

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 194 entries, 2018-04-03 21:00:00+00:00 to 2018-12-30 22:00:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   volume  194 non-null    int64 
 1   open    194 non-null    object
 2   high    194 non-null    object
 3   low     194 non-null    object
 4   close   194 non-null    object
dtypes: int64(1), object(4)
memory usage: 9.1+ KB


In [15]:
# LSTMへの入力用に処理（訓練）
train_lstm_in = []
for i in range(len(train) - window_len):
    temp = train[i:(i + window_len)].copy()
    for col in train:
        temp.loc[:, col] = temp[col] / temp[col].iloc[0] - 1
    train_lstm_in.append(temp)
lstm_train_out = (train['close'][window_len:].values / train['close'][:-window_len].values)-1
 
# LSTMへの入力用に処理（テスト）
test_lstm_in = []
for i in range(len(test) - window_len):
    temp = test[i:(i + window_len)].copy()
    for col in test:
        temp.loc[:, col] = temp[col] / temp[col].iloc[0] - 1
    test_lstm_in.append(temp)
lstm_test_out = (test['close'][window_len:].values / test['close'][:-window_len].values)-1
 

D:\ProgramData\Anaconda3\envs\FX_Evaluation\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\ProgramData\Anaconda3\envs\FX_Evaluation\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\ProgramData\Anaconda3\envs\FX_Evaluation\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [16]:
# PandasのデータフレームからNumpy配列へ変換しましょう
train_lstm_in = [np.array(train_lstm_input) for train_lstm_input in train_lstm_in]
train_lstm_in = np.array(train_lstm_in)
 
test_lstm_in = [np.array(test_lstm_input) for test_lstm_input in test_lstm_in]
test_lstm_in = np.array(test_lstm_in)
 

In [17]:
# Kerasの使用するコンポーネントをインポートしましょう
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout
 
# LSTMのモデルを設定
def build_model(inputs, output_size, neurons, activ_func="linear",
                dropout=0.25, loss="mae", optimizer="adam"):
    model = Sequential()
 
    model.add(LSTM(neurons, input_shape=(inputs.shape[1], inputs.shape[2])))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))
 
    model.compile(loss=loss, optimizer=optimizer)
    return model

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 194 entries, 2018-04-03 21:00:00+00:00 to 2018-12-30 22:00:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   volume  194 non-null    int64  
 1   open    194 non-null    float64
 2   high    194 non-null    float64
 3   low     194 non-null    float64
 4   close   194 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 9.1 KB


In [18]:
# ランダムシードの設定
np.random.seed(202)
 
# 初期モデルの構築
yen_model = build_model(train_lstm_in, output_size=1, neurons = 20)
 
# データを流してフィッティングさせましょう
yen_history = yen_model.fit(train_lstm_in, lstm_train_out, 
                            epochs=50, batch_size=1, verbose=2, shuffle=True)

In [19]:
# 訓練データから予測をして正解レートと予測したレートをプロット
fig, ax1 = plt.subplots(1,1)
ax1.plot(df[df['time']< split_date]['time'][window_len:].astype(datetime),
         train['close'][window_len:], label='Actual', color='blue')
ax1.plot(df[df['time']< split_date]['time'][window_len:].astype(datetime),
         ((np.transpose(yen_model.predict(train_lstm_in))+1) * train['close'].values[:-window_len])[0], 
         label='Predicted', color='red')

In [20]:
# テストデータを使って予測＆プロット
fig, ax1 = plt.subplots(1,1)
ax1.plot(df[df['time']>= split_date]['time'][window_len:].astype(datetime),
         test['close'][window_len:], label='Actual', color='blue')
ax1.plot(df[df['time']>= split_date]['time'][window_len:].astype(datetime),
         ((np.transpose(yen_model.predict(test_lstm_in))+1) * test['close'].values[:-window_len])[0], 
         label='Predicted', color='red')
ax1.grid(True)
 